# Oppretting av svarinngangtabell

### OBS! Denne kjøringen henter ut som default alle data knyttet til ett delregister. Dersom man kun ønsker data for ett (av flere skjema) må det legges til en seleksjon i uttrekket fra DYNAREV

### Hele notebooken kan kjøres på en gang! (Velg "Cell" - "Run All")

In [1]:
import pandas as pd

import sqlite3
from sqlalchemy import Table, create_engine
from sqlalchemy.sql import select
from flask_sqlalchemy import SQLAlchemy

import getpass
import cx_Oracle

from models.models_delt import connect


models_delt.py lastet


/home/jns@ssb.no/edith/models/models_delt.py:6: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt
/home/jns@ssb.no/edith/models/models_delt.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
import json

with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

# Kobler til dynarev / oracle
#### Skriv inn eget Oracle passord

In [5]:
bruker = getpass.getuser()[:3]  # Henter brukernavn direkte
db = cx_Oracle.connect(bruker,getpass.getpass(), "DB1P")  # Kobler til Oracle, skriv inn passord

KeyboardInterrupt: Interrupted by user

In [4]:
conn, engine, db_sqlite = connect()

### Leser og henter inn tabeller fra Dynarev

##### Eks på ekstra betingelse ved seleksjon fra vw_skjema_data: 
f"SELECT ENHETS_ID, FELT_ID, FELT_VERDI FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = {config['perioder']['t']['delreg']}  <b>AND skjema='RA-xxxx'</b>"



In [ ]:
data = pd.read_sql(f"SELECT ENHETS_ID, FELT_ID, FELT_VERDI FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = {config['perioder']['t']['delreg']}", con=db)  # Henter inn faste Oracle variabler
rev = pd.read_sql(f"SELECT ENHETS_ID, REV_KODE, INN_DATO, REV_DATO FROM DYNAREV.VW_SKJEMA_ENHET WHERE DELREG_NR = {config['perioder']['t']['delreg']}", con=db)

In [ ]:
revtabell = data.merge(rev, how='left', on='ENHETS_ID')  # Merger sammen dataframene data og rev med venstrekobling på ENHETS_ID
revtabell['REV_KODE'] = revtabell['REV_KODE'].fillna('U')  # Erstatter NaN verdier med 'U'. Disse merkes dermed som ureviderte
revtabell['FELT_VERDI'] = pd.to_numeric(revtabell['FELT_VERDI'], errors='coerce')  # Konverterer FELT_VERDI til numerisk variabel
revtabell['FELT_VERDI'] = revtabell['FELT_VERDI'].fillna(0)  # Erstatter NaN med 0 i FELT_VERDI
#revtabell.head()

### Laster opp oppdatert tabell

In [ ]:
revtabell.to_sql(f'{config["tabeller"]["svarinngang"]}', con = engine, if_exists='replace')

### Laster inn tabell og sjekker


In [ ]:
revtabell = pd.read_sql("SELECT * FROM svarinngang WHERE FELT_ID = 'INTFOU'", con=engine)

In [ ]:
#revtabell